In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [2]:
query="""
with parent as (select
			sku.sku_id as sku_id,
			po.name as product_option,
			pov.attribute_value as product_option_value_parent
		from
			postgres_broadleaf_federate."broadleaf.blc_sku" sku
			inner join postgres_broadleaf_federate."broadleaf.blc_sku_option_value_xref" x on sku.sku_id = x.sku_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option_value" pov on x.product_option_value_id = pov.product_option_value_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option" po on pov.product_option_id = po.product_option_id
			
		where
			sku.addl_product_id is not null
			and po.name ilike '%-Parent'
			and pov.attribute_value not ilike '%No Aplicable%'
			--and x.sndbx_archived_flag = 'N' and x.sndbx_deleted_flag = 'N' 
			and x.archived = 'N'
			and x.sndbx_id is null
			),
	child as ( select
			sku.sku_id as sku_id,
			po.name as product_option,
			pov.attribute_value as product_option_value_child
		from
			postgres_broadleaf_federate."broadleaf.blc_sku" sku
			inner join postgres_broadleaf_federate."broadleaf.blc_sku_option_value_xref" x on sku.sku_id = x.sku_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option_value" pov on x.product_option_value_id = pov.product_option_value_id
			inner join postgres_broadleaf_federate."broadleaf.blc_product_option" po on pov.product_option_id = po.product_option_id
		where
			sku.addl_product_id is not null
			and po.name not ilike '%-Parent'
			and pov.attribute_value not ilike '%No Aplicable%'
			--and x.sndbx_archived_flag = 'N' and x.sndbx_deleted_flag = 'N' 
			and x.archived = 'N'
			and x.sndbx_id is null
	    )
	    
select distinct 
split_part(bs.upc, '-',1) as region
,bs.sku_id
,bs.date_created fecha_creacion
,split_part(bs.upc, ':',4) as IdProd
,bs.addl_product_id as ID_Tarjeta
,bs.upc
,bp.default_category_id
,bcat2.name as categoria
,CASE
	WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
	WHEN bcat.name ILIKE 'Res%' THEN 'Res'
	WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
	ELSE bcat.name
END AS subcategoria
,fs.owner_id
,fow.name as owner_name
,fbb.name as marca
--bs.name, 
--product_option_value_parent, 
--product_option_value_child, 
,bs2.name as card_name
--,concat(bs2.name,' ', product_option_value_parent,' ' , product_option_value_child) as name
,upper(fp.unit) as unidad
,fp.iva
,fs.step_unit
,fs.min_weight_unit
,fs.conversion_type_x as Type_x
,fs.conversion_value_x as Val_x
,fs.conversion_type_y as Type_y
,fs.conversion_value_y as Val_y
,bs.weight as peso
,bs.weight_unit_of_measure as unidad_peso
,        CASE
            WHEN ((bs.active_end_date IS NULL OR bs.active_end_date > CURRENT_TIMESTAMP) AND (bs2.active_end_date IS NULL OR bs2.active_end_date > CURRENT_TIMESTAMP)) THEN 'on'
            ELSE 'off'
        END as status_frida

from postgres_broadleaf_federate."broadleaf.blc_sku" bs
left join postgres_broadleaf_federate."broadleaf.fb_sku" fs on fs.sku_id = bs.sku_id 
LEFT join child on child.sku_id = bs.sku_id
LEFT join parent on child.sku_id = parent.sku_id
LEFT join postgres_broadleaf_federate."broadleaf.blc_product" bp  on bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_product" fp on fp.product_id = bp.product_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_owner" fow on fow.owner_id = fs.owner_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs2 on bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product_attribute bpa on bpa.product_id = bp.product_id and bpa.name = 'Brand' and bpa.archived = 'N' and bpa.sndbx_tier is null
left join postgres_broadleaf_federate."broadleaf.blc_category_xref" bcx on bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and default_reference = 'true' --Relaciones categorias 
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat on bp.default_category_id = bcat.category_id --Nombre subcategoria
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat2 on bcx.category_id = bcat2.category_id --Nombre categoria
left join postgres_broadleaf_federate."broadleaf.fb_brand" fbb on fbb.brand_id = fp.brand_id

where 
bs.archived = 'N'
and bs.sndbx_id is null
and bp.archived = 'N'
and bs.catalog_disc < 0
and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))"""

frida_products = read_connection_data_warehouse.runQuery(query)

In [3]:
df_margen = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144493,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_margen = pd.merge(df_margen, frida_products,  how='left', left_on=['region','sku'], right_on = ['region','upc'])
df_margen[["gmv_value","cogs_value"]] = df_margen[["gmv_value","cogs_value"]].astype(float)

200


In [4]:
df_margen['day'] = pd.to_datetime(df_margen['day'])
max_index = df_margen.groupby('id_tarjeta')['day'].idxmax()
df_margen = df_margen.loc[max_index].copy()

In [5]:
# calculate the profit margin by sku
df_margen2 = df_margen.groupby(by=['region','id_tarjeta']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum() if x['gmv_value'].sum() != 0 else 0).reset_index(name='GM')

# calculate the profit margin by subcategory
df_margen3 = df_margen.groupby(by=['region','subcategoria']).apply(lambda x: (x['gmv_value'].sum() - x['cogs_value'].sum()) / x['gmv_value'].sum()).reset_index(name='GM')

/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [6]:
query = """
WITH exploded_data AS (
SELECT DISTINCT
    bs.site_identifier_value AS region_code,
    bo.offer_id,
    MAX(bcs.customer_segment_id) AS customer_segment_id_test,
    --bcs.customer_segment_name,
    MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
    --bcs2.customer_segment_name,
    bo.offer_name AS discount,
    DATE(bo.start_date) AS start_date,
    DATE(bo.end_date) AS end_date,
    bo.offer_value::float as Value_Discount,
    replace(
        substring(
          boic.order_item_match_rule, 
          position('["' in boic.order_item_match_rule) + 2, 
          position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
    ), '"', '') AS values_string
    
FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON bcs.customer_segment_name || '_CONTROL' = bcs2.customer_segment_name

WHERE offer_name ILIKE 'SH\_ELAS%'
    AND end_date >= DATE_TRUNC('day', date(getdate()) - interval '9 day')
    --AND end_date >= DATE_TRUNC('month', date(getdate()) - interval '1 month')
    --AND start_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
    AND bo.offer_discount_type = 'PERCENT_OFF'
    AND bo.offer_type = 'ORDER_ITEM'
    AND bo.sndbx_id is NULL
    AND (bo.archived is NULL OR bo.archived='N')
    AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
    AND bo.automatically_added = 'true'
    AND bo.offer_id <> -14489998
    --AND bs.site_identifier_value = 'BOG'
GROUP BY 1,2,5,6,7,8,9
), 
 
number_series AS (
  SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10
  UNION ALL SELECT 11 UNION ALL SELECT 12 UNION ALL SELECT 13 UNION ALL SELECT 14 UNION ALL SELECT 15 UNION ALL SELECT 16 UNION ALL SELECT 17 UNION ALL SELECT 18 UNION ALL SELECT 19 UNION ALL SELECT 20
  UNION ALL SELECT 21 UNION ALL SELECT 22 UNION ALL SELECT 23 UNION ALL SELECT 24 UNION ALL SELECT 25 UNION ALL SELECT 26 UNION ALL SELECT 27 UNION ALL SELECT 28 UNION ALL SELECT 29 UNION ALL SELECT 30
  UNION ALL SELECT 31 UNION ALL SELECT 32 UNION ALL SELECT 33 UNION ALL SELECT 34 UNION ALL SELECT 35 UNION ALL SELECT 36 UNION ALL SELECT 37 UNION ALL SELECT 38 UNION ALL SELECT 39 UNION ALL SELECT 40
  UNION ALL SELECT 41 UNION ALL SELECT 42 UNION ALL SELECT 43 UNION ALL SELECT 44 UNION ALL SELECT 45 UNION ALL SELECT 46 UNION ALL SELECT 47 UNION ALL SELECT 48 UNION ALL SELECT 49 UNION ALL SELECT 50
  UNION ALL SELECT 51 UNION ALL SELECT 52 UNION ALL SELECT 53 UNION ALL SELECT 54 UNION ALL SELECT 55 UNION ALL SELECT 56 UNION ALL SELECT 57 UNION ALL SELECT 58 UNION ALL SELECT 59 UNION ALL SELECT 60
  UNION ALL SELECT 61 UNION ALL SELECT 62 UNION ALL SELECT 63 UNION ALL SELECT 64 UNION ALL SELECT 65 UNION ALL SELECT 66 UNION ALL SELECT 67 UNION ALL SELECT 68 UNION ALL SELECT 69 UNION ALL SELECT 70
  UNION ALL SELECT 71 UNION ALL SELECT 72 UNION ALL SELECT 73 UNION ALL SELECT 74 UNION ALL SELECT 75 UNION ALL SELECT 76 UNION ALL SELECT 77 UNION ALL SELECT 78 UNION ALL SELECT 79 UNION ALL SELECT 80
  UNION ALL SELECT 81 UNION ALL SELECT 82 UNION ALL SELECT 83 UNION ALL SELECT 84 UNION ALL SELECT 85 UNION ALL SELECT 86 UNION ALL SELECT 87 UNION ALL SELECT 88 UNION ALL SELECT 89 UNION ALL SELECT 90
  UNION ALL SELECT 91 UNION ALL SELECT 92 UNION ALL SELECT 93 UNION ALL SELECT 94 UNION ALL SELECT 95 UNION ALL SELECT 96 UNION ALL SELECT 97 UNION ALL SELECT 98 UNION ALL SELECT 99 UNION ALL SELECT 100
)

SELECT DISTINCT
    ed.region_code,
    ed.offer_id,
    ed.customer_segment_id_test,
    ed.customer_segment_id_control,
    ed.discount,
    ed.start_date,
    ed.end_date,
    ed.value_discount,
    bs.addl_product_id AS card_id,
    bs2.name,
    bcat2.name AS cat,
    bcat.name as subcat
    
FROM exploded_data                                                                  ed
JOIN number_series                                                                  ns      ON ns.n <= regexp_count(ed.values_string, ',') + 1 AND trim(split_part(ed.values_string, ',', ns.n)) <> ''
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = trim(split_part(ed.values_string, ',', ns.n))::int
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                     bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id

WHERE
    -- FIX SUPER DESCUENTOS
    bcat2.category_id NOT IN ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_offers = read_connection_data_warehouse.runQuery(query)
df_offers['start_date'] = pd.to_datetime(df_offers['start_date'])
df_offers['end_date'] = pd.to_datetime(df_offers['end_date'])

In [7]:
df_offers_grouped = df_offers.groupby(by = ['region_code', 'offer_id', 'customer_segment_id_test',
       'customer_segment_id_control', 'discount', 'start_date', 'end_date',
       'value_discount']).agg({'card_id': lambda x: x.unique().tolist(), 'name': lambda x: x.unique().tolist()}).reset_index()

In [8]:
df_offers_grouped

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name
0,BAQ,320121,257901,257902,SH_ELASTICITY_2023-08-03_BAQ_Azúcar en Bulto x...,2023-08-04,2023-08-11,5.00,[67822],[Azúcar en Bulto x 50kg ]
1,BAQ,320122,257905,257903,SH_ELASTICITY_2023-08-03_BAQ_Harina de trigo L...,2023-08-04,2023-08-11,1.25,[96574],[Harina de trigo La Nieve 50kg]
2,BAQ,320123,257904,257906,SH_ELASTICITY_2023-08-03_BAQ_Servilletas corta...,2023-08-04,2023-08-11,1.00,[141513],[Servilletas cortadas Mia x 200 und ]
3,BAQ,320124,257907,257908,SH_ELASTICITY_2023-08-03_BAQ_Servilleta cortad...,2023-08-04,2023-08-11,1.25,[438274],[Servilleta cortada Nube x 300 und]
4,BAQ,320125,257909,257910,SH_ELASTICITY_2023-08-03_BAQ_Gaseosa Coca Cola...,2023-08-04,2023-08-11,3.75,[184217],[Gaseosa Coca Cola pet 12und x 400 ml]
...,...,...,...,...,...,...,...,...,...,...
111,VCP,320156,257772,257773,SH_ELASTICITY_2023-08-03_VCP_Batata Pré Frita ...,2023-08-04,2023-08-10,8.75,[427628],[Batata Pré Frita Tradicional 2kg - Mais Batata]
112,VCP,320157,257774,257775,SH_ELASTICITY_2023-08-03_VCP_Filé de Peito de ...,2023-08-04,2023-08-11,13.75,[413514],[Filé de Peito de Frango - Bello Alimentos]
113,VCP,320158,257778,257779,SH_ELASTICITY_2023-08-03_VCP_Filé de Peito de ...,2023-08-04,2023-08-10,2.50,[457069],[Filé de Peito de Frango s/ Pele s/ Osso - Bel...
114,VCP,320159,257776,257777,SH_ELASTICITY_2023-08-03_VCP_Tomate Italiano,2023-08-04,2023-08-10,5.00,[328623],[Tomate Italiano]


In [9]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [10]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment" ba          ON ba.order_id = bo.order_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('year', date(getdate()) - interval '0 year')
    AND bo.submit_date >= DATE_TRUNC('week', date(getdate()) - interval '2 week')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    --AND bo.customer_id IN {clientes}
)

SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    DATE(bo.submit_date) AS submit_date,
    bo.order_id,
    bs.addl_product_id AS padre_sku_id,
    bcat2.name AS cat,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    bs2.name,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75
    ELSE (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776 END AS gmv_usd,    
    CASE 
        WHEN s.site_identifier_value = 'CMX' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'GDL' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'PBC' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'MTY' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/19.65
        WHEN s.site_identifier_value = 'SPO' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'BHZ' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'CWB' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
        WHEN s.site_identifier_value = 'VCP' then (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/4.75
    ELSE (boida.adjustment_value * boipd.quantity * foi.step_unit)*1.0/3776 END AS discount_usd,
    (boi.quantity*foi.step_unit) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
"""#.format(clientes=tuple(set(clear_buyers[clear_buyers.registered_city == city].customer_id.unique())), city = city)

df_sells = read_connection_data_warehouse.runQuery(query)
df_sells[["gmv_usd", "cant","discount_usd"]] = df_sells[["gmv_usd", "cant","discount_usd"]].astype(float)
df_sells['submit_date'] = pd.to_datetime(df_sells['submit_date']) 

In [11]:
print(df_sells.submit_date.min(),df_sells.submit_date.max())

2023-07-24 00:00:00 2023-08-11 00:00:00


In [12]:
def df_info_filter_offer(df_offers_grouped, i, test, control):
    df_filter_test = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(test)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
    df_filter_control = df_sells[(df_sells['city'] == df_offers_grouped.loc[i,'region_code']) & 
                         (df_sells['customer_id'].isin(control)) & 
                         (df_sells['submit_date'] >= df_offers_grouped.loc[i,'start_date']) & 
                         (df_sells['submit_date'] < df_offers_grouped.loc[i,'end_date']) & 
                         (df_sells['padre_sku_id'].isin(df_offers_grouped.loc[i,'card_id']))].copy()
    
    if len(df_filter_control) == 0:
        dic = {}
        dic = df_filter_test.iloc[0].to_dict()
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_control = pd.concat([df_filter_control, new_df], ignore_index=True)

    if len(df_filter_test) == 0:
        dic = {}
        dic = df_filter_control.iloc[0].to_dict()
        dic['customer_id'] = 0
        dic['gmv_usd'] = 0
        dic['discount_usd'] = 0
        dic['cant'] = 0
        new_df = pd.DataFrame(dic, index=[0])
        df_filter_test = pd.concat([df_filter_test, new_df], ignore_index=True)
        
    df_filter_test['group'] = 'test'
    df_filter_control['group'] = 'control'
    df_filter_test['all_customer'] = len(test)
    df_filter_control['all_customer'] = len(control)
    df_filter = pd.concat([df_filter_test, df_filter_control]).reset_index(drop=True)
    df_filter['offer_id'] = df_offers_grouped.loc[i,'offer_id']
    df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: 0.1 if len(df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()) == 0 else df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    #df_filter['GM'] = df_filter.padre_sku_id.apply(lambda x: df_margen2.loc[df_margen2.id_tarjeta == x]['GM'].unique()[0])
    df_filter['gmv_usd_GM'] = df_filter.gmv_usd * df_filter.GM
    
    return df_filter

In [23]:
df_offers_grouped = df_offers_grouped.loc[~df_offers_grouped.offer_id.isin([320115,320119,320091,320069,320180])].reset_index(drop = True).copy()

In [24]:
global_results_list = [] 
columns = ['city', 'offer_id', 'subcat', 'group', 'customer_count', 'gmv_usd',
       'discount_usd', 'quantity', 'all_customer',
       'convertion_rate', 'AOV', 'dct_avg']
results_df = pd.DataFrame()#data={col: [] for col in columns})

for i in range(len(df_offers_grouped)):
    print(df_offers_grouped.loc[i,'offer_id'])
    test = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_test'])
    control = customers_offer(df_offers_grouped.loc[i,'customer_segment_id_control'])
    
    info_offer = df_info_filter_offer(df_offers_grouped, i, test, control)
    results_offer = (info_offer.groupby(by=['city', 'offer_id', 'subcat', 'group', 'all_customer'])
                               .agg(customer_count=('customer_id', 'nunique'),
                                    gmv_usd=('gmv_usd', np.sum),
                                    discount_usd=('discount_usd', np.sum),
                                    quantity=('cant', np.sum),
                                    margen=('GM', np.mean),
                                    cash_margin=('gmv_usd_GM', np.sum))
                               .reset_index()
                    )
    
    results_offer['convertion_rate'] = results_offer.customer_count / results_offer.all_customer       
    results_offer['AOV'] = results_offer.gmv_usd / results_offer.customer_count
    results_offer['AOV_net'] = results_offer.cash_margin / results_offer.customer_count
    results_offer['dct_avg'] = results_offer.discount_usd / results_offer.customer_count
    results_offer['net_cash_margin'] = results_offer.cash_margin - results_offer.discount_usd

    
    results_df = pd.concat([results_df, results_offer]).reset_index(drop=True)

320121
320122
320123
320124
320125
320126
320127
320128
320129
320130
320131
320132
320133
320134
320135
320136
320106
320107
320108
320109
320110
320111
320112
320113
320114
320116
320117
320118
320120
320076
320077
320078
320079
320080
320081
320082
320083
320084
320085
320086
320087
320088
320089
320090
320060
320061
320062
320063
320064
320065
320066
320067
320068
320070
320071
320072
320073
320074
320075
320161
320162
320163
320164
320165
320166
320167
320168
320169
320170
320171
320172
320173
320174
320175
320176
320137
320138
320139
320144
320146
320149
320152
320186
320092
320093
320094
320095
320096
320097
320098
320099
320100
320101
320102
320103
320104
320105
320141
320142
320143
320147
320148
320150
320153
320154
320155
320156
320157
320158
320159
320160


In [25]:
df_offers_grouped

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,discount,start_date,end_date,value_discount,card_id,name
0,BAQ,320121,257901,257902,SH_ELASTICITY_2023-08-03_BAQ_Azúcar en Bulto x...,2023-08-04,2023-08-11,5.00,[67822],[Azúcar en Bulto x 50kg ]
1,BAQ,320122,257905,257903,SH_ELASTICITY_2023-08-03_BAQ_Harina de trigo L...,2023-08-04,2023-08-11,1.25,[96574],[Harina de trigo La Nieve 50kg]
2,BAQ,320123,257904,257906,SH_ELASTICITY_2023-08-03_BAQ_Servilletas corta...,2023-08-04,2023-08-11,1.00,[141513],[Servilletas cortadas Mia x 200 und ]
3,BAQ,320124,257907,257908,SH_ELASTICITY_2023-08-03_BAQ_Servilleta cortad...,2023-08-04,2023-08-11,1.25,[438274],[Servilleta cortada Nube x 300 und]
4,BAQ,320125,257909,257910,SH_ELASTICITY_2023-08-03_BAQ_Gaseosa Coca Cola...,2023-08-04,2023-08-11,3.75,[184217],[Gaseosa Coca Cola pet 12und x 400 ml]
...,...,...,...,...,...,...,...,...,...,...
106,VCP,320156,257772,257773,SH_ELASTICITY_2023-08-03_VCP_Batata Pré Frita ...,2023-08-04,2023-08-10,8.75,[427628],[Batata Pré Frita Tradicional 2kg - Mais Batata]
107,VCP,320157,257774,257775,SH_ELASTICITY_2023-08-03_VCP_Filé de Peito de ...,2023-08-04,2023-08-11,13.75,[413514],[Filé de Peito de Frango - Bello Alimentos]
108,VCP,320158,257778,257779,SH_ELASTICITY_2023-08-03_VCP_Filé de Peito de ...,2023-08-04,2023-08-10,2.50,[457069],[Filé de Peito de Frango s/ Pele s/ Osso - Bel...
109,VCP,320159,257776,257777,SH_ELASTICITY_2023-08-03_VCP_Tomate Italiano,2023-08-04,2023-08-10,5.00,[328623],[Tomate Italiano]


In [26]:
offers_metrics_df = pd.merge(df_offers_grouped[['region_code', 'offer_id','discount', 'start_date', 'end_date','value_discount']],
                          results_df[['offer_id', 'subcat','group', 'customer_count', 'gmv_usd','discount_usd', 'quantity', 'all_customer', 'convertion_rate', 'AOV', 'AOV_net','dct_avg','margen','cash_margin','net_cash_margin']],
                          left_on='offer_id', right_on='offer_id')#.drop(columns = ['id_tarjeta'])

In [27]:
def calculate_ROI(df):
    df.columns = df.iloc[7].unique()
    margen_subcat = df_margen3.loc[(df_margen3.region == df['test'].loc['region_code']) & (df_margen3.subcategoria == df['test'].loc['subcat'])].GM.unique()[0]
    
    GMV_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['AOV_net']# * margen_subcat
    dct_t = df['test'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['test'].loc['dct_avg']

    GMV_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['AOV_net']# * margen_subcat
    dct_c = df['control'].loc['convertion_rate'] * df['test'].loc['all_customer'] * df['control'].loc['dct_avg']
    
    net_GMV_t = GMV_t - dct_t
    net_GMV_c = GMV_c - dct_c
    
    diff_net_GMV = net_GMV_t - net_GMV_c
    diff_net_dct = dct_t - dct_c
    
    ROI = diff_net_GMV / diff_net_dct if diff_net_dct != 0 else 0
    return diff_net_GMV, diff_net_dct, ROI, margen_subcat

In [28]:
for offer in offers_metrics_df.offer_id.unique():
    print(offer)
    diff_net_GMV, diff_net_dct, ROI, margen_subcat = calculate_ROI(offers_metrics_df.loc[offers_metrics_df.offer_id == offer].T)
    offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_GMV'] = diff_net_GMV
    offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'diff_net_dct'] = diff_net_dct
    offers_metrics_df.loc[offers_metrics_df.offer_id == offer,'ROI'] = ROI

320121
320122
320123
320124
320125
320126
320127
320128
320129
320130
320131
320132
320133
320134
320135
320136
320106
320107
320108
320109
320110
320111
320112
320113
320114
320116
320117
320118
320120
320076
320077
320078
320079
320080
320081
320082
320083
320084
320085
320086
320087
320088
320089
320090
320060
320061
320062
320063
320064
320065
320066
320067
320068
320070
320071
320072
320073
320074
320075
320161
320162
320163
320164
320165
320166
320167
320168
320169
320170
320171
320172
320173
320174
320175
320176
320137
320138
320139
320144
320146
320149
320152
320186
320092
320093
320094
320095
320096
320097
320098
320099
320100
320101
320102
320103
320104
320105
320141
320142
320143
320147
320148
320150
320153
320154
320155
320156
320157
320158
320159
320160


In [29]:
offers_metrics_df[['region_code','offer_id','discount','value_discount','group','convertion_rate','customer_count','gmv_usd','discount_usd','dct_avg','cash_margin','net_cash_margin','ROI']]

,region_code,offer_id,discount,value_discount,group,convertion_rate,customer_count,gmv_usd,discount_usd,dct_avg,cash_margin,net_cash_margin,ROI
0,BAQ,320121,SH_ELASTICITY_2023-08-03_BAQ_Azúcar en Bulto x...,5.00,control,0.401130,142,17953.519383,716.388239,5.044988,936.490274,220.102035,-0.172617
1,BAQ,320121,SH_ELASTICITY_2023-08-03_BAQ_Azúcar en Bulto x...,5.00,test,0.501408,178,31978.304507,1599.400063,8.985394,1668.050174,68.650111,-0.172617
2,BAQ,320122,SH_ELASTICITY_2023-08-03_BAQ_Harina de trigo L...,1.25,control,0.338843,41,4737.785495,47.377812,1.155556,251.272526,203.894714,2.280915
3,BAQ,320122,SH_ELASTICITY_2023-08-03_BAQ_Harina de trigo L...,1.25,test,0.442623,54,8035.450249,100.443035,1.860056,426.167011,325.723976,2.280915
4,BAQ,320123,SH_ELASTICITY_2023-08-03_BAQ_Servilletas corta...,1.00,control,0.306220,64,626.135045,28.730617,0.448916,73.491988,44.761371,1.083020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,VCP,320158,SH_ELASTICITY_2023-08-03_VCP_Filé de Peito de ...,2.50,test,0.236842,9,301.473683,7.524210,0.836023,37.353286,29.829076,-2.524204
218,VCP,320159,SH_ELASTICITY_2023-08-03_VCP_Tomate Italiano,5.00,control,0.276730,44,401.593682,15.778945,0.358612,5.718717,-10.060228,-0.887112
219,VCP,320159,SH_ELASTICITY_2023-08-03_VCP_Tomate Italiano,5.00,test,0.300000,48,461.094735,23.065261,0.480526,6.566015,-16.499245,-0.887112
220,VCP,320160,SH_ELASTICITY_2023-08-03_VCP_Batata Pré-Frita ...,6.25,control,0.600000,3,191.115789,4.008421,1.336140,21.565416,17.556995,22.137411


In [34]:
df_offers_grouped.to_csv('SH_info offers.csv', index=False)
offers_metrics_df.to_csv('RESULTS_SH_ROI.csv', index=False)